# Download fMoW - WILDS Version
Use the [WILDS package](https://wilds.stanford.edu/get_started/) to retrieve the fMoW dataset.

In [1]:
from wilds import get_dataset

/home/erik/.conda/envs/wilds/lib/python3.13/site-packages/outdated/__init__.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


To download the dataset, ssl certification needed to be disabled, because the certificate expired. 

```python
import ssl

ssl._create_default_https_context = ssl._create_unverified_contex
dataset = get_dataset(dataset="fmow", download=True)
```

Also in order to load the dataset with `get_dataset`, the time-date format of the metadata needed to be changed manually to drop millisecond data:
```
sed -i 's/\([0-9]\{2\}:[0-9]\{2\}:[0-9]\{2\}\)\.[0-9]\+\(Z\)/\1\2/' rgb_metadata.csv
```

With the result that:
```
2007-01-21T02:38:01.080Z -> 2007-01-21T02:38:01Z
```

In [3]:
dataset = get_dataset(dataset="fmow")